# Batch processing of pressure data

In [13]:
#import libraries
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

In [14]:
# to extract required data from collected samples
def extract_data(path):
    global time_stamps
    global node_list
    global pressure_list
    with open(path, 'r') as file:
        unfiltered_data = file.read()

    unfiltered_data = unfiltered_data.split('[')
    filtered_data = []

    for value in unfiltered_data:
        filtered_data.append(value.split(']'))

    filtered_data = filtered_data[1:]
    final_data = []
    
    for i in range(1, len(filtered_data), 4):
        for value in filtered_data[i]: 
            if value.strip():
                node_values = re.findall(r'\d+', value)
                node = [int(x) for x in node_values]

        for value in filtered_data[i+1]:
            if value.strip():
                pressure_values = re.findall(r'\d+', value)
                pressure = [int(x) for x in pressure_values]

        for value in filtered_data[i+2]:
            value = value.strip()
            
            if (i+2 == len(filtered_data)-1):
                value = value.split('\n')[0]
            if value:
                time = float(value)

        extract = list(zip(node, pressure))
        final_data.append([extract, time])

    return final_data


In [15]:
# init dataframe
def make_df(data):

    num_nodes = 64
    time_list = [entry[1] for entry in data]

    df = pd.DataFrame(0, index = time_list, columns = range(1,num_nodes+1))

    for i, [val,t] in enumerate(data):
        for n,p in val:
            df.at[time_list[i], n] = p

    return df

In [16]:
# retiming and interpolating data
def custom_retime_linear(df, new_time):
    # Convert unix code to DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index, unit = 's')
        
    new_time_series = pd.date_range(start = df.index[0], end = df.index[len(df)-1], freq=new_time)

    # init new df
    new_df = pd.DataFrame(index=new_time_series)

    # Linear interpol
    for column in df.columns:
        new_df[column] = np.interp(new_time_series.astype(np.int64), df.index.astype(np.int64), df[column])
        new_df = new_df.astype(int)

    return new_df


In [17]:
# mapping to RGB values
def map_2_rgb(x):
    if x <= 255:
        return [x, 0, 0] 
    elif x > 255 and x <= 65535:
        return [0, x//256, 0]
    else:
        return [0, 0, x//65536]
        

In [ ]:
# path of directory; modify accordingly
dir = r"D:\iHT\pressure data processing\240523 ctl_5-40"
target_dir = r"D:\iHT\pressure data processing\pressure_data_csv"
animation_dir = r"D:\iHT\pressure data processing\pressure_data_vid"
f_names = os.listdir(dir)
file_paths = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f))]
file_name = os.path.basename(file_paths[0])


for path in file_paths:

    # file names
    file_name = os.path.basename(path)
    file_stem = os.path.splitext(file_name)[0]
    csv_file_path = new_file_path = os.path.join(target_dir, f"{file_stem}.csv")
    animation_file_path = os.path.join(animation_dir, f"{file_stem}.gif")

    dat = extract_data(path)
    df = make_df(dat)
    
    time_difference = pd.Timedelta(seconds=0.0165)
    newdf = custom_retime_linear(df, time_difference)
    newdf.to_csv(csv_file_path, header='timestamp', index_label='nodes')

    fig, ax = plt.subplots()
    im = ax.imshow(np.zeros((8, 8, 3), dtype=int)) 

    def update(frame):
        pressure_data = newdf.iloc[frame]
        rgb_data = np.array([[map_2_rgb(value) for value in row] for row in pressure_data.values.reshape((8, 8))])
        im.set_array(rgb_data)
        return im


    ani = FuncAnimation(fig, update, frames=range(len(newdf)), interval=30)
    plt.axis('off')
    ani.save(animation_file_path, writer='pillow')
    plt.close()

